In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

from scipy.stats import multivariate_normal

from utils import *

# Lecture 2.2: Markov Chain Monte Carlo

* Metropolis-Hastings algorithm
* Gibbs sampling
* Hybrid Monte Carlo

MacKay, **Chapter 29** and **Chapter 30**.

_Recommended readings_:

* [Timothy Budd](https://hef.ru.nl/~tbudd/) has an excellent course on Monte Carlo techniques - lecture material is available as a Jupyter book [here](https://hef.ru.nl/~tbudd/mct/intro.html).
* [A Conceptual Introduction to Hamiltonian Monte Carlo](https://arxiv.org/abs/1701.02434) by [Michael Betancourt](https://betanalpha.github.io/)
* [MCMC using Hamiltonian dynamics](https://arxiv.org/abs/1206.1901) and [Bayesian Learning for Neural Networks](https://glizen.com/radfordneal/ftp/thesis.pdf) by [Radford Neal](https://glizen.com/radfordneal/)
* [Stan](https://mc-stan.org/docs/stan-users-guide/index.html) user guide
* An informed [post](https://statmodeling.stat.columbia.edu/2025/10/03/its-a-jax-jax-jax-jax-world/#:~:text=Like%20Stan%2C%20JAX%20is%20also,it's%20wonderfully%20compositional%20and%20general.&text=As%20much%20as%20people%20like,to%20just%20write%20JAX%20directly.) on the current state of affairs in Bayesian modeling: [Stan](https://mc-stan.org/) vs [JAX](https://github.com/jax-ml/jax) [vs [PyMC](https://www.pymc.io/welcome.html) vs [NumPyro](https://github.com/pyro-ppl/numpyro)]

# Metropolis-Hastings algorithm

Suppose we would like to sample from a distribution $p(x) = p^*(x)/Z$.

The Metropolis-Hastings (MH) algorithm is composed of two steps:
* **propose** a state $x'$ using a distribution $q(x'|x^r)$ that generally depends on the current sample $x^r$
* **accept** the state $x'$ with probability
  $$p_{a}\left(x',x^r\right)=\min\left(1,a\left(x',x^r\right)\right)$$
  $$a\left(x',x^{r}\right)=\frac{p^{*}\left(x'\right)q\left(x^{r}|x'\right)}{p^{*}\left(x^{r}\right)q\left(x'|x^{r}\right)}$$

  If $x'$ is accepted:
  $$x^{r+1}=x'$$
  else
  $$x^{r+1}=x^r$$

### Why MH works

We would like to prove that MH transition satisfies DB, i.e.


Consider two states $x$, $x'$ and note that $a\left(x',x\right)=\frac{1}{a\left(x,x'\right)}$

Suppose $a_{x'x}\ge 1$. Then:

  * Given $x$ the probability to accept $x'$ is
    $$T(x'|x)=q(x'|x)$$
  * Given $x'$ the probability to accept $x$ is
    $$T(x|x')=q(x|x')a_{xx'}$$
    $$\frac{T(x'|x)}{T(x|x')}=a_{x'x}\frac{q(x'|x)}{q(x|x')}=\frac{p^*(x')}{p^*(x)} =\frac{p(x')}{p(x)},$$

The MH algorithm is thus an ergodic (aperiodic) Markov process that satisfies detailed balance: the process $T(x'|x)$ converges to $p(x)$.

# Gibbs sampling

At each time step, update a single variable $i$ using its conditional distribution from the target distribution $p(x)$.

$$p(x_i'|x_{\setminus i})$$

keeping all other elements $x'_{\setminus i}$ unchanged [$x_{\setminus i}$ is the vector of variables without $x_i$].

A Gibbs sampling step is a MH step with acceptance $a=1$. Indeed the update of the $x\to x'$ is described by
$$q(x'|x)=p(x'_i|x_{\setminus i})\delta_{x_{\setminus i},x'_{\setminus i}}$$
and for the acceptance:
$$a=\frac{p(x')}{p(x)}\frac{q(x|x')}{q(x'|x)}=\frac{p(x_i'|x'_{\setminus i})p(x'_{\setminus i})}{p(x_i|x_{\setminus i})p(x_{\setminus i})}\frac{p(x_i|x'_{\setminus i})}{p(x'_i|x_{\setminus i})}=\frac{p(x_i'|x_{\setminus i})}{p(x_i|x_{\setminus i})}\frac{p(x_i|x_{\setminus i})}{p(x'_i|x_{\setminus i})}=1$$
where we used $x'_{\setminus i}=x_{\setminus i}$.

### *Bonus: MH vs Gibbs in a discrete space*

Let us take a distribution of the usual form
$$p(x)=\frac{1}{Z}\exp \left(- E(x)\right)$$
with $\left\{ -1,1\right\} ^{N}$. Let us introduce the flip operator $F_i$, i.e. $F_i x = (x_{\setminus i}, -x_i)$.

Given state $x=(x_{\setminus i},x_i)$, the probability to go to state $x'=(x_{\setminus i},-x_i)$ is
$$\text{MH}:~~~p(x'|x)=\text{min}\left(e^{-\Delta E_{x,F_i x}},1\right)$$
$$\text{Gibbs}:~~~p(x'|x)=p(-x_i|x_{\setminus i})=\frac{p(-x_i,x_{\setminus i})}{p(x_{\setminus i})}=\frac{e^{-E(-x_i,x_{\setminus i})}}{e^{-E(x_i,x_{\setminus i})}+e^{-E(-x_i,x_{\setminus i})}}=\frac{1}{1+e^{\Delta E_{x,F_i x}}}$$

Both transitions respect detailed balance:
$$\text{MH}:~~~\frac{p(x'|x)}{p(x|x')}=\frac{\min\left(e^{-\Delta E},1\right)}{\min\left(e^{\Delta E},1\right)}=e^{-\Delta E}=\frac{p(x')}{p(x)}$$

$$\text{Gibbs}:~~~\frac{p(x'|x)}{p(x|x')}=\frac{p(-x_i|x_{\setminus i})}{p(x_i|x_{\setminus i})}=\frac{1+e^{-\Delta E}}{1+e^{\Delta E}}=e^{-\Delta E}=\frac{p(x')}{p(x)}$$

### Correlations slow down sampling

When $q(x'|x)$ is Gaussian centered on $x$, $\frac{q(x'|x)}{q(x|x')}$ independent of $x,x'$:
$$a_{x'x}=\frac{p^*(x')}{p^*(x)}$$

**$\epsilon$ large:**
Acceptance rate $a_{x'x}$ small.

**$\epsilon$ small:**
Strong dependence on starting value.
Many samples needed to sample.

With step $\epsilon$ random, the particle moves a distance $L \approx \sqrt{T} \epsilon$ in $T$ iterations. If largest length scale is $L$ then
$$T\approx \left(\frac{L}{\epsilon}\right)^2$$
time steps are needed to sample the length $L$.

Convergence slows down when variables are correlated: you will experiment with this problem in **Ex 2.1**.

# Hybrid Monte Carlo

### In theory

The idea behing Hybrid (or Hamiltonian) Monte Carlo (HMC) is to use the information in $\nabla_x \log p(x)$ to reduce the diffusive (random walk) behaviour of the MH method.

Consider a distibution $p$ on an $N$-dimensional space
$$p(x)=\frac{e^{-E(x)}}{Z}$$
where $E$ and its gradient $\frac{\partial E}{\partial x_i}$ are easy to compute.

For each $x_i$, introduce a *momentum* $p_i$ variable, thus doubling the state space to $2N$ variables.

Define the *Hamiltonian*
$$H(p,x)=E(x)+\frac{\alpha}{2}\sum_i p_i^2$$
and a new distribution in the enlarged space
$$P_H(p,x)=\frac{1}{Z_H}\exp\left(-E(x)-\frac{\alpha}{2}\sum_i p_i^2\right)$$

Sampling from $P_H$ can be accomplished by using Hamilton equations:
$$\frac{d p_i}{d t}=-\frac{\partial H}{\partial x_i} \qquad \frac{d x_i}{d t}=\frac{\partial H}{\partial p_i}$$
which leave $H$ invariant:

$$\frac{dH}{dt}=\sum_i \frac{dH}{dp_i}\frac{dp_i}{dt}+\frac{dH}{dx_i}\frac{dx_i}{dt}=0$$

The original distribution $p$ can be obtained by marginalizing over the momentum variables:
$$p(x)=\int dp P_H(p,x)$$

In practice any numerical integration will deviate from the true Hamiltonian flow - even when using *simplectic integrators* like **leapfrog**.

We therefore use the resulting configuratin from Hamiltonian dynamics as a proposal and accept it using the MH protocol.

# Hybrid Monte Carlo

### In practice

Choose initial point $x^1$.

For t up to T

1.  choose $p^t$ from $\mathcal{N}(0,\alpha^{-1})$, yielding $(x^t,p^t)$
2.  run Hamilton dynamics using Leapfrog, yielding $(x',p')$
3.  accept $(x^{t+1},p^{t+1})=(x',p')$ as new state using MH:
    $$\mathsf{min}\left(1,a\right)\qquad a=\frac{P_H(x',p')}{P_H(x^t,p^t)}=\frac{e^{-H(x',p')}}{e^{-H(x^t,p^t)}}$$
4.  On rejection, $(x^{t+1},p^{t+1})=(x^t,p^t)$

As said earlier, although the acceptance probability is $a=1$ in theory, it deviates from $1$ due to integration errors.

#### Leapfrog

Calling 
$$g_{t}=\nabla_{x}H\left(x_{t}\right)$$
the gradient of $H$ wrt the $x$ variables, the update equations for the leapfrog integrators with integration parameter $\epsilon$ are the following:

$$p_{t+\frac{1}{2}}=p_{t}-\frac{\epsilon}{2}g_{t}$$
$$x_{t+1}=x_{t}+\epsilon p_{t+\frac{1}{2}}$$
$$p_{t+1}=p_{t+\frac{1}{2}}-\frac{\epsilon}{2}g_{t+1}$$

# An example of HMC

Consider the double well cost $E(x)=\left(x^2-1\right)^2$.

In [ ]:
xs = np.linspace(-2, 2, 100)
Es = (xs**2 - 1)**2
gradEs = (4 * xs * (xs**2 - 1))
plt.plot(xs, Es, label='E', color='black');
plt.plot(xs, -gradEs, label='-dE/dx', c='red', alpha=0.5);
plt.hlines(y=0, xmin=-2, xmax=2, ls=':', color='gray')
plt.xlabel('x')
plt.xlabel('E, dE/dx')
plt.ylim([-10,10])
plt.legend();

In [ ]:
# run a Langevin dynamics
num_iter = 10000
dt = 0.1
sigma = 0.5 # try small sigma, like 0.1

sqdt = np.sqrt(dt)
xstory = np.zeros(num_iter)

x = -1.
for t in range(num_iter):
    xstory[t] = x
    x = (1-dt) * x - dt * (4 * x * (x**2 - 1)) + sigma * np.random.randn() * sqdt

plt.figure(figsize=(10,4))

plt.subplot(121)
plt.plot(xstory);

plt.subplot(122)
plt.plot(xs, Es, label='E');
plt.hist(xstory, bins=100, alpha=0.5, density=True);

plt.tight_layout();

The Hamiltonian $H(x,p)=E(x) +\frac{1}{2} \alpha p^2$ has the form.

The parameter $\alpha$ plays the roll of inverse inertia: small $\alpha$ will yield large $p$ values and dynamical trajectories travelling large distances.

In [ ]:
alpha = 1
invsqalpha = 1. / np.sqrt(alpha)
ps = np.linspace(-2, 2, 100)
xgrid, pgrid = np.meshgrid(xs, ps)
Hgrid = (xgrid**2 - 1)**2 + 0.5 * alpha * pgrid**2

plt.imshow(Hgrid, origin="lower", extent=[-2, 2, -2, 2], alpha=0.8, cmap="viridis");
plt.contour(xgrid, pgrid, Hgrid);
plt.xlabel('x')
plt.ylabel('p')
plt.title('H');

Here's an example of four subsequent (accepted) proposals in a HMC with 10 leapfrog steps and integration constant `eps = 0.1`. In the assignment, you will get to implement HMC and observe such trajectories.

In [ ]:
xstories_alpha1 = np.loadtxt("data/xstories_alpha1.dat")
pstories_alpha1 = np.loadtxt("data/pstories_alpha1.dat")

# visualize_stories
for it in range(4):
    plt.plot(xstories_alpha1[it,0], pstories_alpha1[it,0], '.', alpha=0.6, c=color_cycle[it]);
    plt.plot(xstories_alpha1[it,-1], pstories_alpha1[it,-1], 'x', alpha=0.6, c=color_cycle[it]);
    plt.plot(xstories_alpha1[it,:], pstories_alpha1[it,:], alpha=0.6, c=color_cycle[it]);
plt.xlabel('x')
plt.ylabel('p');
plt.title('H');

plt.imshow(Hgrid, origin="lower", extent=[-2, 2, -2, 2], alpha=0.1, cmap="viridis");
plt.contour(xgrid, pgrid, Hgrid, alpha=0.1);

# <center>Assignments</center>

#### Ex 2.1

In this exercise, you will study the effect of strong correlations between variables on the efficiency of MCMC.

Consider the 2-dimensional zero-mean Gaussian distribution with the following covariance matrix:

$$\left(\begin{array}{cc}
1 & 0.998\\
0.998 & 1
\end{array}\right)$$

1. Write a computer program to sample from this distribution using the Metropolis Hastings algorithm. Study the acceptance ratio and how well the sampler covers the entire distribution by varying the width $\sigma$ of an isotropic Gaussian proposal distribution. Report the optimal values.
2. Write a computer program to sample from this distribution using the Hamilton Monte Carlo algorithm. Study the acceptance ratio and how well the sampler covers the entire distribution by varying the step size $\epsilon$ and the number of leap frog steps $\tau$. Report the optimal values.
3. Compute the mean of this Gaussian distribution using both methods and compare the accuracy as a function of the computation time.

#### Ex 2.2

##### Setting

Consider the learning problem for logistic regression as explained in **Lecture1.1** and in MacKay chapters **39** and **41**. Our aim is to sample from the distribution $p\left(w|D, α\right)$ as given by MacKay Eqs. 41.8-10. Let us recap it in the following (note: the notation is slightly different from the book).

**Data set**:
$$D=\{x^\mu,t^\mu\}$$ 


with $t^\mu=0, 1$ and $\mu=1,\ldots,P$.

**Conditional model** (probability of label $t$ given $x$):

$$p(t=1|x,w)=\sigma(w_0+w_1 x_1 +w_2 x_2)$$
$$\sigma(h)=\frac{1}{1+\exp(-h)}$$

**Likelihood**:
$$p(D|w)=\prod_\mu p(t^\mu|x^\mu,w)=e^{-\mathcal{E}(w)}\qquad \mathcal{E}(w)=-\sum_\mu \log p(t^\mu|x^\mu,w)$$

**Prior**:
$$p(w)=\frac{e^{-R(w,\alpha)}}{Z_w(\alpha)}\qquad R(w,\alpha)=\alpha \sum_i w_i^2$$

**Posterior**:
$$p(w|D)=\frac{p(D|w)p(w)}{p(D)} \propto e^{-L(w)}$$
with
$$L(w)=\mathcal{E}(w)+R(w)$$

##### Exercise:

Implement MH and Hamiltonian Monte Carlo to sample from the distribution $p\left(w|D, α\right)$ (experiment with various values of $\alpha$. For both methods, reproduce plots similar to MacKay Fig. 41.5 and estimate the burn in time that is required before the sampler reaches the equilibrium distribution.

Investigate the acceptance ratio for both methods and try to optimize this by varying the proposal distribution, the step size in HMC and the number of leap frog steps $\tau$.

Use the 10 2-dimensional inputs in `X` and their associated binary targets `t` in the following cell.

In [ ]:
# dataset to be used
X = np.array([[2., 3.], [3., 2.], [3., 6.], [5.5, 4.5], [5., 3.], [7., 4.], [5., 6.], [8., 6.], [9.5, 5.], [9., 7.]])
t = np.array([0., 0., 0., 0., 0., 1., 1., 1., 1., 1.])

plot_points(X, t, plt)

#### Ex 2.3

Consider the Algorithm 41.4 in MacKay, in the section *The Langevin Monte Carlo method*.

Explain why such implementation uses a single leapfrog step, as per the caption: *To obtain the Hamiltonian Monte Carlo method, we repeat the four lines marked * multiple times (algorithm 41.8)*.

Discuss the relation with the [Metropolis-adjusted Langevin algorithm](https://en.wikipedia.org/wiki/Metropolis-adjusted_Langevin_algorithm) (MALA) and the need of an appropriate choice of `epsilon` for the leapfrog step.